## Load Libraries

In [4]:
import pandas as pd
import numpy as np

## Useful functions

In [5]:
# Transform hours to seconds
def hoursToSec(s, factors=(1, 60, 3600)): 
    """[[hours:]minutes:]seconds -> seconds"""
    return int(sum(x*y for x, y in zip(map(float, s.split(':')[::-1]), factors)))

In [6]:
# Transform seconds to minutes
def secToMins(d):
    return d/60.

In [7]:
# Transform minutes to calories
def minToCalories(d):
    constant = 10.83 #assumed calories burnt per minute
    return d*constant

In [8]:
# Add daily burnt to daily default need calories
def sumCalories(d):
    constant = 2495 #calories needed without activity
    return d+constant

## Convert Biking minutes to calories burnt 

I have to mention that it was impossible to download the raw data from the **LifeSum** so I manually created the day-by-day csv. Furthermore, the **Google Fit** app that it was supposed to sync the biking hours with the **LifeSum** app, didn't work for all the days, and when it worked, it didn't add the correct biking time distance of the day, so this data was also manually added to the app from the **FitBit** data.

In [9]:
# Load the data into a Dataframe using pandas
path = 'python_data/calories.csv'
df = pd.read_csv(path)

In [10]:
df[:5]

,date,day_no,hours,consumed,water
0,20170705,1,03:03:23,2559,1.5
1,20170706,2,06:16:52,2336,2.5
2,20170707,3,06:43:24,1714,2.0
3,20170708,4,06:06:41,3458,2.5
4,20170709,5,06:02:04,3175,2.5


In [11]:
# Convert hours to seconds and minutes columns
df['secs']=df['hours'].map(lambda x: hoursToSec(x))
df['mins']=df['secs'].map(lambda x:secToMins(x))

In [12]:
df[:5]

,date,day_no,hours,consumed,water,secs,mins
0,20170705,1,03:03:23,2559,1.5,11003,183.383333
1,20170706,2,06:16:52,2336,2.5,22612,376.866667
2,20170707,3,06:43:24,1714,2.0,24204,403.400000
3,20170708,4,06:06:41,3458,2.5,22001,366.683333
4,20170709,5,06:02:04,3175,2.5,21724,362.066667


On average, a 180 lb. cyclist (81kg as I am) riding at a moderate effort level will burn approximately **650 calories per hour** (10.83 calories per minute). This number is a rough estimate; however, the precise number depends on a number of factors, not all of which are easy to quantify. So we will calculate the calories burnt based on this assumption. Check [this](http://www.ilovebicycling.com/how-many-calories-do-you-burn-when-cycling/) link for further details.

In [13]:
# Convert mins to calories burnt
df['burnt']=df['mins'].map(lambda x: minToCalories(x))

In [14]:
df[:5]

,date,day_no,hours,consumed,water,secs,mins,burnt
0,20170705,1,03:03:23,2559,1.5,11003,183.383333,1986.0415
1,20170706,2,06:16:52,2336,2.5,22612,376.866667,4081.4660
2,20170707,3,06:43:24,1714,2.0,24204,403.400000,4368.8220
3,20170708,4,06:06:41,3458,2.5,22001,366.683333,3971.1805
4,20170709,5,06:02:04,3175,2.5,21724,362.066667,3921.1820


According to **LifeSum** and based on my profile metrics, I need 2495 Kcal per day without any activity. So if we sum up this constant to the burnt calories column, we come up to the daily need. 

In [15]:
# Create the need column
df['need']=df['burnt'].map(lambda x: sumCalories(x))

In [16]:
df[:5]

,date,day_no,hours,consumed,water,secs,mins,burnt,need
0,20170705,1,03:03:23,2559,1.5,11003,183.383333,1986.0415,4481.0415
1,20170706,2,06:16:52,2336,2.5,22612,376.866667,4081.4660,6576.4660
2,20170707,3,06:43:24,1714,2.0,24204,403.400000,4368.8220,6863.8220
3,20170708,4,06:06:41,3458,2.5,22001,366.683333,3971.1805,6466.1805
4,20170709,5,06:02:04,3175,2.5,21724,362.066667,3921.1820,6416.1820


In [17]:
# Drop not needed time columns
df.drop(df.columns[[2, 5, 6]], axis=1, inplace=True)

In [18]:
df[:5]

,date,day_no,consumed,water,burnt,need
0,20170705,1,2559,1.5,1986.0415,4481.0415
1,20170706,2,2336,2.5,4081.4660,6576.4660
2,20170707,3,1714,2.0,4368.8220,6863.8220
3,20170708,4,3458,2.5,3971.1805,6466.1805
4,20170709,5,3175,2.5,3921.1820,6416.1820


Now we can source the calories_from_moves dataframe in order to have an extra column to cross check and keep the one that better describes what happened and which makes more sense.

In [19]:
# Read new csv
df_calories_from_moves = pd.read_csv('python_data/calories_from_moves.csv')

In [21]:
# Merge with df
df = pd.merge(df, df_calories_from_moves, on='day_no')

In [25]:
# Drop useless column
df.drop(df.columns[[6]], axis=1, inplace=True)
df[:5]

,date,day_no,consumed,water,burnt,need,ttl_cal_burnt
0,20170705,1,2559,1.5,1986.0415,4481.0415,2105
1,20170706,2,2336,2.5,4081.4660,6576.4660,3471
2,20170707,3,1714,2.0,4368.8220,6863.8220,3830
3,20170708,4,3458,2.5,3971.1805,6466.1805,3760
4,20170709,5,3175,2.5,3921.1820,6416.1820,4273


## Create a csv export for D3 

In [26]:
# Save it to a csv for D3
df.to_csv('../d3_visualizations/calories/calories.csv', index=False)